## Cellpose in Python

[Website](https://www.cellpose.org) | [GitHub](https://github.com/mouseland/cellpose) | [Paper](https://www.biorxiv.org/content/10.1101/2025.04.28.651001v1)

## Overview

This notebook shows how to process your own images saved on Google Drive with [Cellpose](https://www.cellpose.org).

The structure of the notebook is based on the [Cellpose examples](https://github.com/mouseland/cellpose).

## Make sure you have GPU access

To Enable GPU:

1. navigate to `Runtime -> Change Runtime Type`
2. select `Python 3` as `Runtime Type`
3. select one available GPU (e.g. `T4 GPU`) as `Hardware accelerator`.

<br>

<div align="left"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/cellpose/colab_runtime.png" alt="Ilastik Logo" width="400"></div>


## Mount your google drive

To access the data for the course you first need to mount your Google Drive.

Run the cell below to connect your Google Drive to colab and follow the instructions to authenticate your Google account.

You will need to allow access to your Google Drive so that the notebook can read and write files.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")


Then click on `folder icon` on the left bar, press the `refresh button`. Your Google Drive folder should now be available here (e.g. MyDrive).

<div align="left"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/cellpose/colab_folder.png" alt="Ilastik Logo" width="300"></div>

## Download the Data

Run the cell below to download the data for this exercise and save it in you Google Drive. A new folder called `bobiac_data_cellpose` will be created in your Google Drive.

In [ ]:
# Create directory
!mkdir -p /content/bobiac_data_cellpose
# Download the data
!wget https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/data/05_segmentation_cellpose.zip -O /content/bobiac_data_cellpose/05_segmentation_cellpose.zip
# Unzip the data, remove zip file and macOS metadata files (if any)
!cd /content/bobiac_data_cellpose && unzip 05_segmentation_cellpose.zip && rm -f 05_segmentation_cellpose.zip && rm -rf __MACOSX

## Install Cellpose


In [ ]:
# !pip install cellpose


## Import Libraries

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from cellpose import core, io, models, plot
from tqdm import tqdm

## Setup

In [ ]:
io.logger_setup()  # run this to get printing of progress

use_gpu = core.use_gpu()
if use_gpu:
    print("Using GPU for Cellpose")
else:
    raise ImportError("No GPU access, change your runtime as explained above.")

## Run Cellpose on A Sample Image

In this section we will...

### Load the Image

All the data for this exercise is stored in the `bobiac_data_cellpose` folder on your Google Drive created in the previous steps. You can get the path to the files inside this folder by clicking on the `folder icon` on the left bar, then right-clicking on the file and selecting `Copy path`.

In [ ]:
image_path = "/content/bobiac_data_cellpose/05_segmentation_cellpose/cell_cellpose.tif"
image = tifffile.imread(image_path)

print(image.shape)

### Initialize the Model

To initialize Cellpose model we can use the [`models.CellposeModel()`](https://cellpose.readthedocs.io/en/latest/api.html#cellposemodel) class.

Among the parameters we can set if we want to use `gpu` (if available) as well which **model** to use. The current default is Cellpose-SAM as `cpsam` (another example is `cyto3`, the previous U-Net-based model).

If it is the first time you run this notebook, the model will be downloaded automatically. This may take a while.

In [ ]:
model = models.CellposeModel(gpu=use_gpu, model_type="cpsam")

### Run Cellpose

After initializing the model, we can run it on the image using the [`model.eval()`](https://cellpose.readthedocs.io/en/latest/api.html#id0) method.

Here are some of the parameters you can change:

* ***flow_threshold*** is  the  maximum  allowed  error  of  the  flows  for  each  mask.   The  default  is 0.4.
    *  **Increase** this threshold if cellpose is not returning as many masks as you’d expect (or turn off completely with 0.0)
    *   **Decrease** this threshold if cellpose is returning too many ill-shaped masks.

* ***cellprob_threshold*** determines proability that a detected object is a cell.   The  default  is 0.0.
    *   **Decrease** this threshold if cellpose is not returning as many masks as you’d expect or if masks are too small
    *   **Increase** this threshold if cellpose is returning too many masks esp from dull/dim areas.

* ***tile_norm_blocksize*** determines the size of blocks used for normalizing the image. The default is 0, which means the entire image is normalized together.
  You may want to change this to 100-200 pixels if you have very inhomogeneous brightness across your image.


In [ ]:
channel = 0  # channel to use for cell detection, 0 cytoplasm, 1 nucleus

# Cellpose parameters
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0

masks, flows, styles = model.eval(
    image[channel],
    batch_size=32,
    flow_threshold=flow_threshold,
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize},
)

### Display the Results

In [ ]:
fig = plt.figure(figsize=(12, 5))
plot.show_segmentation(fig, image[channel], masks, flows[0])
plt.tight_layout()
# Optional if you want to also save the figure
# fname = Path(image_path).stem + "_cp_output.png"
# plt.savefig(f"/content/bobiac_data_cellpose/05_segmentation_cellpose/{fname}")
plt.show()

## Run Cellpose on a Folder of Images

...

In [ ]:
# folder_path = Path("data/05_segmentation_cellpose")
folder_path = Path("/content/bobiac_data_cellpose/05_segmentation_cellpose")

# Read file names
images = []
for f in folder_path.glob("*.tif"):
    images.append(f)
# same as running: files = [f for f in folder_path.glob("*.tif")]

# Cellpose parameters
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0

# Run Cellpose on all images
for image_path in tqdm(images):
    image = tifffile.imread(image_path)
    masks, flows, styles = model.eval(
        image,
        batch_size=32,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        normalize={"tile_norm_blocksize": tile_norm_blocksize},
    )

    # Optional: display each image with its segmentation
    # fig = plt.figure(figsize=(12, 5))
    # plot.show_segmentation(fig, image, masks, flows[0])
    # plt.tight_layout()
    # plt.show()

    # Save the segmentation results
    output_path = folder_path / f"{image_path.stem}_labeled_mask.tif"
    tifffile.imwrite(output_path, masks.astype("uint16"))